In [1]:
!az login --tenant 16b3c013-d300-468d-ac64-7eda0820b6d3

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "16b3c013-d300-468d-ac64-7eda0820b6d3",
    "id": "781b03e7-6eb7-4506-bab8-cf3a0d89b1d4",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47"
      }
    ],
    "name": "MCAPS-Hybrid-REQ-66223-2023-antonslutsky",
    "state": "Enabled",
    "tenantId": "16b3c013-d300-468d-ac64-7eda0820b6d3",
    "user": {
      "name": "antonslutsky@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "16b3c013-d300-468d-ac64-7eda0820b6d3",
    "id": "4e54a795-4951-418e-af5b-79baae7ecf07",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47"
      }
    ],
    "name": "alzeltov-mcap-nonprod",
    "state": "Enabled",
    "tenantId": "16b3c013-d300-468d-ac64-7eda0820b6d3",
    "user": {
      "name": "antonslutsky@microsoft.com",
      "type": "user"
    }
  }
]


In [2]:
!az configure --defaults workspace=gpu-workspace group=antonslutsky-rg

In [7]:
%%writefile ./dependencies/job.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ../src
command: >-
  python mae_trainer.py 
  --dataset_directory_name ${{inputs.data}} 
  --trainset ${{inputs.trainset}} 
  --valset ${{inputs.valset}} 
  --num_epochs ${{inputs.num_epochs}} 
  --batch_size ${{inputs.batch_size}} 
  --num_workers ${{inputs.num_workers}}  
  --num_nodes ${{inputs.num_nodes}} 
  --num_devices ${{inputs.num_devices}}  
  --strategy ${{inputs.strategy}} 
  --img_size ${{inputs.img_size}} 
  --learning_rate ${{inputs.learning_rate}} 
  --weight_decay ${{inputs.weight_decay}}
inputs:
  data:
    type: uri_folder
    path: azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/antonslutsky-rg/workspaces/gpu-workspace/datastores/workspaceblobstore/paths/UI/2023-12-05_185853_UTC/ImageNet_1000_sm/imagenet-mini/
    mode: ro_mount      
  trainset:
    type: uri_file
    path: azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/antonslutsky-rg/workspaces/gpu-workspace/datastores/workspaceblobstore/paths/UI/2023-12-05_190730_UTC/ImageNet_1000_train.csv
    mode: ro_mount
  valset:
    type: uri_file
    path: azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/antonslutsky-rg/workspaces/gpu-workspace/datastores/workspaceblobstore/paths/UI/2023-12-05_190801_UTC/ImageNet_1000_val.csv
    mode: ro_mount         

  num_epochs: 500
  batch_size: 129
  num_workers: 3
  num_nodes: 1
  num_devices: 1
  strategy: deepspeed_stage_1
  img_size: 224
  learning_rate: 5e-4
  weight_decay: 0.05

environment: azureml:godzilla-environment@latest
resources:
  instance_count: 3
  shm_size: 1000G
distribution:
  type: pytorch 
  process_count_per_instance: 1
compute: azureml:minizilla-1xV100x4-cluster
display_name: mae_trainer
experiment_name: mae_trainer
description: Train a Machine Learning model using a workspace Data asset.

Overwriting ./dependencies/job.yml


In [8]:
!az ml job create -f dependencies/job.yml

{
  "code": "azureml:/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServices/workspaces/gpu-workspace/codes/91975b91-8110-4db3-86cc-ffada893e430/versions/1",
  "command": "python mae_trainer.py  --dataset_directory_name ${{inputs.data}}  --trainset ${{inputs.trainset}}  --valset ${{inputs.valset}}  --num_epochs ${{inputs.num_epochs}}  --batch_size ${{inputs.batch_size}}  --num_workers ${{inputs.num_workers}}   --num_nodes ${{inputs.num_nodes}}  --num_devices ${{inputs.num_devices}}   --strategy ${{inputs.strategy}}  --img_size ${{inputs.img_size}}  --learning_rate ${{inputs.learning_rate}}  --weight_decay ${{inputs.weight_decay}}",
  "compute": "azureml:minizilla-1xV100x4-cluster",
  "creation_context": {
    "created_at": "2023-12-11T14:27:27.362731+00:00",
    "created_by": "Anton Slutsky",
    "created_by_type": "User"
  },
  "description": "Train a Machine Learning model using a workspace Data asset.",
  "displa

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
